In [ ]:
from google.colab import drive

drive.mount('/content/drive')
root_dir = "/content/drive/My Drive/"
wiki_dir = root_dir + 'Tesi/Wiki50/Wikipedia/'

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
import nltk.tokenize.mwe
from nltk import tokenize
from nltk import pos_tag, word_tokenize

In [ ]:
!pip install stanza

In [ ]:
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline

In [ ]:
import re

In [ ]:
punctuations = ''''"“”’'<>–~'''

WIKIPEDIA DUMP

In [ ]:
wiki_dir = root_dir + 'Tesi/Wiki50/Wikipedia/'

In [ ]:
with open(wiki_dir + 'wikipedia_list_preprocessed.txt') as f:
    wikipedia_list = f.read().splitlines() 

In [ ]:
wikipedia_list

WIKI50

In [ ]:
base_dir = root_dir + 'Tesi/Wiki50/'

In [ ]:
import pandas as pd

wiki_df = pd.read_csv(base_dir + 'wiki50_preprocessed.csv', converters={'result': eval})

ENTITY EXTRACTION

In [ ]:
wiki_df["entities"]=""

In [ ]:
def entities_extraction(text):
    doc = nlp(text)

    entities= [ent.text for sentence in doc.sentences for ent in sentence.entities if ent.type in {"PERSON", "ORG", "GPE", "NORP", "FAC", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART", "LAW", "LANGUAGE", "MISC"}]

    return entities

In [ ]:
wiki_df["entities"] = wiki_df['text'].apply(lambda x: entities_extraction(x))

In [ ]:
entities_without_unigrams = []

In [ ]:
for i in range(len(wiki_df)):
  for kw in wiki_df['entities'][i]:
    if(' ' in kw):
        kw = kw.lower()
        entities_without_unigrams.append(kw)
  wiki_df['entities'][i]=entities_without_unigrams
  entities_without_unigrams = []

In [ ]:
entities_without_article = []

In [ ]:
for i in range(len(wiki_df)):
  for kw in wiki_df['entities'][i]:
    if('the' in kw[:4]):
        kw = re.sub("the ", "", kw)
        entities_without_article.append(kw)
    else:
        entities_without_article.append(kw)
  wiki_df['entities'][i]=entities_without_article
  entities_without_article = []

In [ ]:
for s in range(len(wiki_df)):  
  for i in range(len(wiki_df['entities'][s])):
    wiki_df['entities'][s][i] = wiki_df['entities'][s][i].split()

In [ ]:
#remove entities overlapping 

for s in range(len(wiki_df)):  
  for i in range(len(wiki_df['entities'][s])-1,-1,-1):
    if (wiki_df['entities'][s][i][0] == wiki_df['entities'][s][i-1][-1]):
      wiki_df['entities'][s][i-1].remove(wiki_df['entities'][s][i-1][-1])
      wiki_df['entities'][s][i] = wiki_df['entities'][s][i-1] + wiki_df['entities'][s][i]
      wiki_df['entities'][s].remove(wiki_df['entities'][s][i-1])

In [ ]:
for s in range(len(wiki_df)):   
  for i in range(len(wiki_df['entities'][s])):
    wiki_df['entities'][s][i] = ' '.join(wiki_df['entities'][s][i])

In [ ]:
wiki_df

POS

In [ ]:
wiki_df["pos"]=""

In [ ]:
def extract_phrases(my_tree, phrase):
   my_phrases = []
   if my_tree.label() == phrase:
      my_phrases.append(my_tree.copy(True))

   for child in my_tree:
       if type(child) is nltk.Tree:
            list_of_phrases = extract_phrases(child, phrase)
            if len(list_of_phrases) > 0:
                my_phrases.extend(list_of_phrases)

   return my_phrases

In [ ]:
grammar = "WP: {<NN><NN.?><NN.?><NN|NNS>|<NN|NNS><IN><DT>?<NN>|<NN><POS><NN|NNS>|<JJ><NN|NNS>+|<NN|NNS><IN><JJ><NN|NNS>|<IN><NN|NNS>|<NN|NNS><CC><NN|NNS>|<NN|NNS><NN>?<NN>?<NN|NNS>|<JJ>+<NN|NNS>}"

cp = nltk.RegexpParser(grammar)

In [ ]:
def pos_extraction(x):
    pos_list=[]

    sentence = pos_tag(tokenize.word_tokenize(x))
    tree = cp.parse(sentence)
    list_of_noun_phrases = extract_phrases(tree, 'WP')
    for phrase in list_of_noun_phrases:
      pos_list.append(" ".join([x[0] for x in phrase.leaves()]))

    text=pos_list
    
    return text

In [ ]:
wiki_df["pos"] = wiki_df['text'].apply(lambda x: pos_extraction(x))

In [ ]:
pos_without_unigrams = []

In [ ]:
for i in range(len(wiki_df)):
  for kw in wiki_df['pos'][i]:
    if(' ' in kw):
        kw = kw.lower()
        pos_without_unigrams.append(kw)
  wiki_df['pos'][i]=pos_without_unigrams
  pos_without_unigrams = []

In [ ]:
for s in range(len(wiki_df)):  
  for i in range(len(wiki_df['pos'][s])):
    wiki_df['pos'][s][i] = wiki_df['pos'][s][i].split()

In [ ]:
# remove overlapping mwe

for s in range(len(wiki_df)):  
  for i in range(len(wiki_df['pos'][s])-1,-1,-1):
    if (wiki_df['pos'][s][i][0] == wiki_df['pos'][s][i-1][-1]):
      wiki_df['pos'][s][i-1].remove(wiki_df['pos'][s][i-1][-1])
      wiki_df['pos'][s][i] = wiki_df['pos'][s][i-1] + wiki_df['pos'][s][i]
      wiki_df['pos'][s].remove(wiki_df['pos'][s][i-1])

In [ ]:
for s in range(len(wiki_df)):   
  for i in range(len(wiki_df['pos'][s])):
    wiki_df['pos'][s][i] = ' '.join(wiki_df['pos'][s][i])

In [ ]:
wiki_df

PREPROCESS

In [ ]:
for i in range(len(wiki_df)):
  wiki_df['pos'][i] = list(dict.fromkeys(wiki_df['pos'][i]))

In [ ]:
for i in range(len(wiki_df)):
  wiki_df['entities'][i] = list(dict.fromkeys(wiki_df['entities'][i]))

In [ ]:
for i in range(len(wiki_df)):
  wiki_df['result'][i] = list(dict.fromkeys(wiki_df['result'][i]))

In [ ]:
wiki_df["united"] = wiki_df["pos"] + wiki_df["entities"]

In [ ]:
for i in range(len(wiki_df)):
  wiki_df['united'][i] = list(dict.fromkeys(wiki_df['united'][i]))

In [ ]:
wiki_df

WIKIPEDIA COMPARISON

In [ ]:
wiki_df['check'] = ""

In [ ]:
wiki_df["check"] = [set(wikipedia_list) & set(wiki_df.loc[r, "united"]) for r in range(len(wiki_df))]

SAVE DATAFRAME

In [ ]:
# if you want to save dataframe

compression_opts = dict(method='zip', archive_name='wiki50Patterns_mwe.csv')

wiki_df.to_csv('wiki50Patterns_mwe.zip', index=False, compression=compression_opts)

MATCHES

In [ ]:
matches_df = pd.DataFrame(columns=['united']) 

In [ ]:
#test
matches_df["united"] = [set(wiki_df.loc[r, "result"]) & set(wiki_df.loc[r, "united"]) for r in range(len(wiki_df))]

In [ ]:
#test
matches_df["entities"] = [set(wiki_df.loc[r, "result"]) & set(wiki_df.loc[r, "entities"]) for r in range(len(wiki_df))]

In [ ]:
matches_df["check"] = [set(wiki_df.loc[r, "result"]) & set(wiki_df.loc[r, "check"]) for r in range(len(wiki_df))]

In [ ]:
matches_df

In [ ]:
result_list = wiki_df['result'].tolist()

In [ ]:
count_result = []

for i in range(len(result_list)):
  for el in result_list[i]:
    count_result.append(el)

CHECK PRECISION & RECALL

In [ ]:
checkMatch_list = list(map(list, matches_df['check']))

In [ ]:
count_checkMatch = []

for i in range(len(checkMatch_list)):
  for el in checkMatch_list[i]:
    count_checkMatch.append(el)

In [ ]:
check_list = wiki_df['check'].tolist()

In [ ]:
count_check = []

for i in range(len(check_list)):
  for el in check_list[i]:
    count_check.append(el)

In [ ]:
recall = len(count_checkMatch)/len(count_result) #recall

In [ ]:
precision = len(count_checkMatch)/len(count_check) #precision

In [ ]:
fscore = 2 * (precision * recall)/(precision + recall)

In [ ]:
recall

In [ ]:
precision

In [ ]:
fscore